<form action="https://nbviewer.jupyter.org/github/prmiles/pyfod/blob/master/tutorials/index.ipynb">
    <input type="submit" value="Return to Index" style="background-color: green; color: white; width: 150px; height: 35px; float: right"/>
</form>

# Caputo Fractional Derivative

Author(s): Graham T. Pash, Paul R. Miles | May 26, 2019

The Caputo definition for a fractional derivative with order $\alpha \in [0, 1)$ is
$$
D^\alpha[f(t)] = \frac{1}{\Gamma(1-\alpha)}\int_{t_0}^{t}\frac{f^\prime(s)}{(t-s)^\alpha}ds.
$$
we observe that $D^\alpha[f(t)]$ by applying a finite-difference scheme to approximate the integer order derivative and applying a quadrature scheme to handle the integration. That is, the Caputo definition can be approximated as:
$$
D^\alpha[f(t)] \approx \frac{1}{\Gamma(1-\alpha)}\sum\limits_{j=1}^N\frac{f(t_j)-f(t_{j-1})}{t_j - t_{j-1}}\cdot\frac{1}{(t_j-s)^\alpha}.
$$

For details on how this expression is numerically approximated, users are referred to the following publication(s):
- Miles, P. R., Pash, G. T., Oates, W. S., Smith, R. C. (2018, September). Numerical Techniques to Model Fractional-Order Nonlinear Viscoelasticity in Soft Elastomers. In ASME 2018 Conference on Smart Materials, Adaptive Structures and Intelligent Systems (pp. V001T03A021). American Society of Mechanical Engineers. http://dx.doi.org/10.1115/SMASIS2018-8102

**Note:** This method is currently limited for problems where the fractional order is $\alpha \in [0,1)$.

# Example:
We consider the function
$$f(t) = \exp(2t)$$

In [1]:
import numpy as np
def f(t):
    return np.exp(2*t)

where $t \in [0, 1]$ and $\alpha = 0.9$.  Using [Mathematica](https://www.wolfram.com/mathematica/), we find that
$$
\frac{1}{\Gamma(1-0.9)}\int_0^1\frac{\frac{d}{dt}\exp(2s)}{(1-s)^{0.9}}ds = \frac{1}{\Gamma(1-0.9)}\int_0^1\frac{2\exp(2s)}{(1-s)^{0.9}}ds = 13.7102.
$$
**Note**: The Riemann-Liouville and Caputo definitions of the fractional derivative are equivalent only under certain conditions. Thus, for the same test function, we obtain a different value.

The user may select from any of the quadrature methods available in `pyfod`. For this example, a Riemann-Sum quadrature scheme is employed.

In [2]:
from pyfod.fod import caputo
alpha = 0.9
out = caputo(f=f, alpha=alpha, lower=0, upper=1, quadrature='rs')
print('D^{}[exp(2t)] = {}'.format(alpha, out['fd']))

D^0.9[exp(2t)] = 11.237433548776709


This is a poor approximation of the fractional derivative.  To improve the approximation, we can increase the grid-resolution of the quadrature method.

In [3]:
from time import time

gs = 2**np.arange(3, 11, 1)
exact = 13.7102
fd = []
error = []
runtime = []
nruns = 100
print('Convergence using Gauss-Legendre quadrature:')
for n in gs:
    st = time()
    for ii in range(nruns):
        if ii == 0:
            fd.append(caputo(f=f, alpha=0.9, lower=0, upper=1, quadrature='rs', n=n)['fd'])
        else:
            caputo(f=f, alpha=0.9, lower=0, upper=1, quadrature='rs', n=n)
    et = time()
    error.append(np.abs((exact - fd[-1]))/exact)
    runtime.append((et - st)/nruns)
    print('n = {}, D^{}[exp(2t)] = {}'.format(n, alpha, fd[-1]))

# store output
out = dict(rs=dict(gs=gs, fd=np.array(fd), error=np.array(error),
                   runtime=np.array(runtime),
                   plot=dict(marker='s', color='r', linewidth=2,
                            markersize=10, mfc='none',
                            label='Riemann-Sum')))

Convergence using Gauss-Legendre quadrature:
n = 8, D^0.9[exp(2t)] = 12.298395538265186
n = 16, D^0.9[exp(2t)] = 13.073968990978361
n = 32, D^0.9[exp(2t)] = 13.417767556595134
n = 64, D^0.9[exp(2t)] = 13.574199254577481
n = 128, D^0.9[exp(2t)] = 13.646294966078516
n = 256, D^0.9[exp(2t)] = 13.679730777857426
n = 512, D^0.9[exp(2t)] = 13.695284637679043
n = 1024, D^0.9[exp(2t)] = 13.702530788775583
